In [1]:
import math
import numpy as np
import scipy.io as sio
import os
import tqdm
from numpy import linalg as LA
import pickle
import time
import torch
from statistics import mean
import scipy.io as sio

In [4]:
def data_to_vector(data_for_trial, L_mean_29k, L_std_29k, R_mean_29k, R_std_29k, factor=(2/1.65)):
    
    L_Icoint_to_value = {}
    R_Icoint_to_value = {}
    
    
    
    for i in range(40962):
    
        list_of_Euclideans = Icoint_to_Euclideans[i]
    
        L_val_list = []
        R_val_list = []
    
        for j in range(len(list_of_Euclideans)):

            x,y = list_of_Euclideans[j]

            L_val_list.append(data_for_trial[x][y].item())

            R_val_list.append(data_for_trial[x][y+131].item())

        L_Icoint_to_value[i] = mean(L_val_list) * factor
        R_Icoint_to_value[i] = mean(R_val_list) * factor
    
    
    
    L_fake_data = np.zeros(29696)

    R_fake_data = np.zeros(29716)

    for i in range(len(L_fake_data)):
    
        (Ldis0, Lnum0), (Ldis1, Lnum1), (Ldis2, Lnum2) = L_int_to_ico_three[i]
    
        Ltotal = Ldis0+Ldis1+Ldis2
    
        L_fake_data[i]=(Ldis0*L_Icoint_to_value[Lnum2]+Ldis1*L_Icoint_to_value[Lnum1]+Ldis2*L_Icoint_to_value[Lnum0])/Ltotal   
      
    
    
    for i in range(len(R_fake_data)):
    
        (Rdis0, Rnum0), (Rdis1, Rnum1), (Rdis2, Rnum2) = R_int_to_ico_three[i]
    
        Rtotal = Rdis0+Rdis1+Rdis2
    
        R_fake_data[i]=(Rdis0*R_Icoint_to_value[Rnum2]+Rdis1*R_Icoint_to_value[Rnum1]+Rdis2*R_Icoint_to_value[Rnum0])/Rtotal   
                
            
    #L_fake_data = L_fake_data*L_std_29k.reshape((len(L_std_29k)))+L_mean_29k.reshape((len(L_mean_29k)))
    
    #R_fake_data = R_fake_data*R_std_29k.reshape((len(R_std_29k)))+R_mean_29k.reshape((len(R_mean_29k)))
    
    LR_fake_data = np.concatenate([L_fake_data, R_fake_data])
    
    return LR_fake_data

In [5]:
# make a dict that corresponds a coordinate in the atlas to that of an icosahedron

phi = (1+ math.sqrt(5))/2
scale = 100 / math.sqrt(1+phi**2)
atlas_to_coordinate = {}

atlas_to_coordinate[(0,0,0)] = scale * np.array([-1,0,phi])
atlas_to_coordinate[(0,64,0)] = scale * np.array([-phi, -1,0])
atlas_to_coordinate[(0,128,0)] = scale * np.array([-1,0,-phi])
atlas_to_coordinate[(0,0,64)] = scale * np.array([1,0,phi])
atlas_to_coordinate[(0,64,64)] = scale * np.array([0,-phi,1])
atlas_to_coordinate[(0,128,64)] = scale * np.array([0,-phi,-1])

atlas_to_coordinate[(1,0,0)] = scale * np.array([0,phi,1])
atlas_to_coordinate[(1,64,0)] = scale * np.array([-phi,1,0])
atlas_to_coordinate[(1,128,0)] = scale * np.array([-1,0,-phi])
atlas_to_coordinate[(1,0,64)] = scale * np.array([1,0,phi])
atlas_to_coordinate[(1,64,64)] = scale * np.array([-1,0,phi])
atlas_to_coordinate[(1,128,64)] = scale * np.array([-phi, -1,0])

atlas_to_coordinate[(2,0,0)] = scale * np.array([phi,1,0])
atlas_to_coordinate[(2,64,0)] = scale * np.array([0,phi,-1])
atlas_to_coordinate[(2,128,0)] = scale * np.array([-1,0,-phi])
atlas_to_coordinate[(2,0,64)] = scale * np.array([1,0,phi])
atlas_to_coordinate[(2,64,64)] = scale * np.array([0,phi,1])
atlas_to_coordinate[(2,128,64)] = scale * np.array([-phi,1,0])

atlas_to_coordinate[(3,0,0)] = scale * np.array([phi,-1,0])
atlas_to_coordinate[(3,64,0)] = scale * np.array([1,0,-phi])
atlas_to_coordinate[(3,128,0)] = scale * np.array([-1,0,-phi])
atlas_to_coordinate[(3,0,64)] = scale * np.array([1,0,phi])
atlas_to_coordinate[(3,64,64)] = scale * np.array([phi,1,0])
atlas_to_coordinate[(3,128,64)] = scale * np.array([0,phi,-1])

atlas_to_coordinate[(4,0,0)] = scale * np.array([0,-phi,1])
atlas_to_coordinate[(4,64,0)] = scale * np.array([0,-phi,-1])
atlas_to_coordinate[(4,128,0)] = scale * np.array([-1,0,-phi])
atlas_to_coordinate[(4,0,64)] = scale * np.array([1,0,phi])
atlas_to_coordinate[(4,64,64)] = scale * np.array([phi,-1,0])
atlas_to_coordinate[(4,128,64)] = scale * np.array([1,0,-phi])

In [6]:
# make a dict of triangles; the key is the depth of the icosahedron

triangles = {}

triangles[0] = []

triangles[0].append(((0,0,0),(0,0,64),(0,64,64)))
triangles[0].append(((0,0,0),(0,64,0),(0,64,64)))
triangles[0].append(((0,128,64),(0,64,0),(0,64,64)))
triangles[0].append(((0,64,0),(0,128,0),(0,128,64)))

triangles[0].append(((1,0,0),(1,0,64),(1,64,64)))
triangles[0].append(((1,0,0),(1,64,0),(1,64,64)))
triangles[0].append(((1,128,64),(1,64,0),(1,64,64)))
triangles[0].append(((1,64,0),(1,128,0),(1,128,64)))

triangles[0].append(((2,0,0),(2,0,64),(2,64,64)))
triangles[0].append(((2,0,0),(2,64,0),(2,64,64)))
triangles[0].append(((2,128,64),(2,64,0),(2,64,64)))
triangles[0].append(((2,64,0),(2,128,0),(2,128,64)))

triangles[0].append(((3,0,0),(3,0,64),(3,64,64)))
triangles[0].append(((3,0,0),(3,64,0),(3,64,64)))
triangles[0].append(((3,128,64),(3,64,0),(3,64,64)))
triangles[0].append(((3,64,0),(3,128,0),(3,128,64)))

triangles[0].append(((4,0,0),(4,0,64),(4,64,64)))
triangles[0].append(((4,0,0),(4,64,0),(4,64,64)))
triangles[0].append(((4,128,64),(4,64,0),(4,64,64)))
triangles[0].append(((4,64,0),(4,128,0),(4,128,64)))

In [7]:
# make new triangles while storing data into make_triangles and atlas_to_coordinate

def make_triangles(coord0, coord1, coord2, depth):
    
    atlas_to_coordinate[(coord0[0], int((coord1[1]+coord0[1])/2), int((coord1[2]+coord0[2])/2))] = (atlas_to_coordinate[coord0]+atlas_to_coordinate[coord1])/2
                         
    atlas_to_coordinate[(coord0[0], int((coord2[1]+coord1[1])/2), int((coord2[2]+coord1[2])/2))] = (atlas_to_coordinate[coord2]+atlas_to_coordinate[coord1])/2
                         
    atlas_to_coordinate[(coord0[0], int((coord2[1]+coord0[1])/2), int((coord2[2]+coord0[2])/2))] = (atlas_to_coordinate[coord0]+atlas_to_coordinate[coord2])/2
    
    triangles[depth].append((
        coord0, 
        (coord0[0], int((coord0[1]+coord1[1])/2), int((coord0[2]+coord1[2])/2)), 
        (coord0[0], int((coord2[1]+coord0[1])/2), int((coord2[2]+coord0[2])/2))
    ))     
    
    triangles[depth].append((
        coord1, 
        (coord0[0], int((coord0[1]+coord1[1])/2), int((coord0[2]+coord1[2])/2)), 
        (coord0[0], int((coord2[1]+coord1[1])/2), int((coord2[2]+coord1[2])/2))
    ))   
    
    triangles[depth].append((
        coord2, 
        (coord0[0], int((coord2[1]+coord1[1])/2), int((coord2[2]+coord1[2])/2)), 
        (coord0[0], int((coord2[1]+coord0[1])/2), int((coord2[2]+coord0[2])/2))
    ))   
    
    triangles[depth].append((
        (coord0[0], int((coord2[1]+coord1[1])/2), int((coord2[2]+coord1[2])/2)), 
        (coord0[0], int((coord0[1]+coord1[1])/2), int((coord0[2]+coord1[2])/2)), 
        (coord0[0], int((coord2[1]+coord0[1])/2), int((coord2[2]+coord0[2])/2))
    ))

In [8]:
# make three new triangles for each triangle

def dig_deeper(current_depth):
    
    triangles[current_depth + 1] = []
    
    for triangle in triangles[current_depth]:
        make_triangles(triangle[0], triangle[1], triangle[2], current_depth + 1)
# For our purpose, we want to do this for six times

for i in range(6):
    dig_deeper(i)

In [9]:
set_of_coordinates = set()

for value in atlas_to_coordinate.values():
    
    set_of_coordinates.add(tuple(value))

In [10]:
coordinate_to_int = {}

for i, coordinate in enumerate(set_of_coordinates):
    
    coordinate_to_int[coordinate] = i
    
int_to_coordinate = dict(map(reversed, coordinate_to_int.items()))

In [11]:
# make a dict mapping euclidean coord to atlas coord

Euclidean_to_atlas = {}

for j in range(130):
    for i in range(66):
        
        if j == 0 or i == 65:
            
            Euclidean_to_atlas[(i,j)] = "zero"
        else:
            Euclidean_to_atlas[(i,j)] = (0, j-1, 64-i)
        

for j in range(130):
    for i in range(66,132):
        
        if j == 0 or i == 131:
            Euclidean_to_atlas[(i,j)] = "zero"
            
        else:
            Euclidean_to_atlas[(i,j)] = (1, j-1, 130-i)
        
for j in range(130):
    for i in range(132, 198):
        
        if j == 0 or i == 197:
            Euclidean_to_atlas[(i,j)] = "zero"
            
        else:
            Euclidean_to_atlas[(i,j)] = (2, j-1, 196-i)
        
        
for j in range(130):
    for i in range(198, 264):
        
        if j == 0 or i == 263:
            Euclidean_to_atlas[(i,j)] = "zero"
            
        else:
            Euclidean_to_atlas[(i,j)] = (3, j-1, 262-i)
        
        
for j in range(130):
    for i in range(264, 330):
        
        if j == 0 or i == 329:
            Euclidean_to_atlas[(i,j)] = "zero"
            
        else:
            Euclidean_to_atlas[(i,j)] = (4, j-1, 328-i)
        

In [12]:
Icoint_to_Euclideans = {}
for i in range(40962):
    
    Icoint_to_Euclideans[i] = []
for i in range(330):
        
    for j in range(130):
            
        if Euclidean_to_atlas[(i,j)] != "zero":

            x,y,z = atlas_to_coordinate[Euclidean_to_atlas[(i,j)]]
            
            coordinate = (x,y,z)
            
            Icoint = coordinate_to_int[coordinate]
            
            Icoint_to_Euclideans[Icoint].append((i,j))

In [13]:
# coordinate of 32k L-sphere
L_coord32k_sphere = sio.loadmat("data/Sphere_L_32k.mat")["Lcoordinate"]

R_coord32k_sphere = sio.loadmat("data/Sphere_R_32k.mat")["Rcoordinate"]

L_32k_to_29k = sio.loadmat("data/32k_to_29k.mat")["tem5"]
R_32k_to_29k = sio.loadmat("data/R_32k_to_29k.mat")["tem11"]
# only use 29k
L_coord29k_sphere = L_coord32k_sphere[np.all(L_32k_to_29k == 1, axis = 1)]

R_coord29k_sphere = R_coord32k_sphere[np.all(R_32k_to_29k == 1, axis = 1)]

In [22]:
L_int_to_ico_three = {}
R_int_to_ico_three = {}
for i in tqdm.tqdm(range(len(L_coord29k_sphere))):
    
    
    
    coordinate = L_coord29k_sphere[i]
    

    L_min_three = []



    for j in range(len(int_to_coordinate)):

        # distance between Left coordinate and the icosahedron
        distance = LA.norm(coordinate - int_to_coordinate[j])

        if len(L_min_three) < 3:
            L_min_three.append( (distance, j) )

        else:

            if max(L_min_three)[0] > distance:

                L_min_three.remove(max(L_min_three))

                L_min_three.append( (distance, j) )
    
    
    L_min_three.sort()
    
    
    
    
    
    L_int_to_ico_three[i] = L_min_three

100%|██████████| 29696/29696 [5:43:54<00:00,  1.44it/s]  


In [23]:
for i in tqdm.tqdm(range(len(R_coord29k_sphere))):
    
    coordinate = R_coord29k_sphere[i]
    

    R_min_three = []



    for j in range(len(int_to_coordinate)):

        # distance between Left coordinate and the icosahedron
        distance = LA.norm(coordinate - int_to_coordinate[j])

        if len(R_min_three) < 3:
            R_min_three.append( (distance, j) )

        else:

            if max(R_min_three)[0] > distance:

                R_min_three.remove(max(R_min_three))

                R_min_three.append( (distance, j) )
    
    
    R_min_three.sort()
    
    
    
    
    
    R_int_to_ico_three[i] = R_min_three

100%|██████████| 29716/29716 [5:47:55<00:00,  1.42it/s]  


In [24]:
"""
with open('data/L_int_to_ico_three.pickle', 'wb') as f:
    pickle.dump(L_int_to_ico_three, f)
with open('data/R_int_to_ico_three.pickle', 'wb') as f:
    pickle.dump(R_int_to_ico_three, f)
"""

In [14]:
with open('data/L_int_to_ico_three.pickle', 'rb') as f:
    L_int_to_ico_three = pickle.load(f)

with open('data/R_int_to_ico_three.pickle', 'rb') as f:
    R_int_to_ico_three = pickle.load(f)

In [15]:
import torch

In [16]:
unprocessed_data = sio.loadmat("data/dtseries1200.mat")["ata"]

print(unprocessed_data.shape)


useful_data = np.delete(unprocessed_data, slice(64984,91282),0) 

useful_mean = useful_data.mean(axis=1,keepdims =True)

useful_std = useful_data.std(axis=1,keepdims =True)

L_32k_to_29k = sio.loadmat("data/32k_to_29k.mat")["tem5"]
R_32k_to_29k = sio.loadmat("data/R_32k_to_29k.mat")["tem11"]

L_mean_32k = np.delete(useful_mean, slice(32492,64984),0) 
L_std_32k = np.delete(useful_std, slice(32492,64984),0) 
R_mean_32k = np.delete(useful_mean, slice(0,32492),0) 
R_std_32k = np.delete(useful_std, slice(0,32492),0) 
# only use 29k
L_mean_29k = L_mean_32k[np.all(L_32k_to_29k == 1, axis = 1)]
L_std_29k = L_std_32k[np.all(L_32k_to_29k == 1, axis = 1)]

R_mean_29k = R_mean_32k[np.all(R_32k_to_29k == 1, axis = 1)]
R_std_29k = R_std_32k[np.all(R_32k_to_29k == 1, axis = 1)]

(91282, 1200)


In [17]:
""""
with open('Wasserstein_GAN/fake_data_wgan_9900.pickle','rb')as f:
    fake=pickle.load(f)
"""

'"\nwith open(\'Wasserstein_GAN/fake_data_wgan_9900.pickle\',\'rb\')as f:\n    fake=pickle.load(f)\n'

In [156]:
with open('Wasserstein_GAN/fake_data_wgan.pickle', 'rb') as f:
    fake_data_list = pickle.load(f)

In [157]:
with open('Wasserstein_GAN/data.pickle', 'rb') as f:
    real_data_list = pickle.load(f)

In [158]:
fake = fake_data_list[23][0]
real = real_data_list[23][0]

In [159]:
fake=torch.tensor(fake)
real = torch.tensor(real)

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [160]:
fake.shape

torch.Size([330, 261])

In [161]:
fake_vector = data_to_vector(fake, L_mean_29k, L_std_29k, R_mean_29k, R_std_29k, factor=(2/1.65))

In [162]:
real_vector = data_to_vector(real, L_mean_29k, L_std_29k, R_mean_29k, R_std_29k, factor=(2/1.65))

In [108]:
"""
fake_vector = []
for i in range(len(fake)):
    fake_data = fake[i][0]
    fake_data = torch.tensor(fake_data)
    fake_vector.append(list(data_to_vector(fake_data, L_mean_29k, L_std_29k, R_mean_29k, R_std_29k, factor=(2/1.65))))
"""

'\nfake_vector = []\nfor i in range(len(fake)):\n    fake_data = fake[i][0]\n    fake_data = torch.tensor(fake_data)\n    fake_vector.append(list(data_to_vector(fake_data, L_mean_29k, L_std_29k, R_mean_29k, R_std_29k, factor=(2/1.65))))\n'

In [163]:
import pandas as pd

In [81]:
df = pd.read_csv('visualization/weights/100206encoder_weight_corr0.2291.csv')

In [164]:
df.head()

,0,1,2,3,4,5,6,7
0,-0.759428,-1.185240,-0.813755,-0.285380,-0.913292,-0.635820,-0.850608,-1.088829
1,-0.768449,-0.172972,-0.775843,-0.236510,-0.705354,-0.240639,-0.768416,0.475913
2,0.478434,-2.745289,0.353571,0.685677,-0.064554,0.344520,0.215787,0.923635
3,0.068147,-1.052365,-0.007782,-1.433972,-0.159418,-0.200158,-0.042512,-0.884231
4,-0.801572,0.723356,-0.825801,0.416932,-0.920837,0.415956,-0.872106,0.101925


In [165]:
df.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7'], dtype='object')

In [83]:
df = df.drop(['Unnamed: 0','2','3','4','5','6','7'],axis=1)

In [166]:
df.head()

,0,1,2,3,4,5,6,7
0,-0.759428,-1.185240,-0.813755,-0.285380,-0.913292,-0.635820,-0.850608,-1.088829
1,-0.768449,-0.172972,-0.775843,-0.236510,-0.705354,-0.240639,-0.768416,0.475913
2,0.478434,-2.745289,0.353571,0.685677,-0.064554,0.344520,0.215787,0.923635
3,0.068147,-1.052365,-0.007782,-1.433972,-0.159418,-0.200158,-0.042512,-0.884231
4,-0.801572,0.723356,-0.825801,0.416932,-0.920837,0.415956,-0.872106,0.101925


In [167]:
df['2'] = pd.DataFrame(fake_vector)

In [168]:
df['3'] = pd.DataFrame(real_vector)

In [169]:
df.head()

,0,1,2,3,4,5,6,7
0,-0.759428,-1.185240,-0.902364,-1.063202,-0.913292,-0.635820,-0.850608,-1.088829
1,-0.768449,-0.172972,-0.645394,0.377891,-0.705354,-0.240639,-0.768416,0.475913
2,0.478434,-2.745289,-0.172759,-0.752595,-0.064554,0.344520,0.215787,0.923635
3,0.068147,-1.052365,-0.190476,-1.916845,-0.159418,-0.200158,-0.042512,-0.884231
4,-0.801572,0.723356,-0.930472,1.602748,-0.920837,0.415956,-0.872106,0.101925


In [170]:
df.to_csv('visualization/weights/100206encoder_weight_corr0.2291.csv')